In [28]:
from QuantLib import *

In [29]:
# calendar data
today = Date(1,1,2016)
calendar = UnitedStates()
day_count = ActualActual()

# option data
ref_date = Date(1,1,2016)
tenor = Period(1, Years)
maturity_date = calendar.advance(ref_date, tenor)

spot_price = 1000
strike_price = 1000
option_type = Option.Call
sched = Schedule(ref_date, maturity_date, Period(Monthly),
                 calendar, Following, Following, DateGeneration.Forward, False)

sched_dates = [dt for dt in list(sched) if dt>=today]
times = [day_count.yearFraction(today, dt) for dt in sched_dates]

for date, time in zip(sched_dates, times):
    print("Date: {dt}, Time {t}".format(dt=date, t=time))
    
tg = TimeGrid(times)

# market
risk_free_rate = 0.001
dividend_rate =  0.0163
volatility = .2

calculation_date = today
Settings.instance().evaluationDate = calculation_date



Date: January 4th, 2016, Time 0.00819672131147541
Date: February 1st, 2016, Time 0.08469945355191257
Date: March 1st, 2016, Time 0.16393442622950818
Date: April 1st, 2016, Time 0.24863387978142076
Date: May 2nd, 2016, Time 0.3333333333333333
Date: June 1st, 2016, Time 0.41530054644808745
Date: July 1st, 2016, Time 0.4972677595628415
Date: August 1st, 2016, Time 0.5819672131147541
Date: September 1st, 2016, Time 0.6666666666666666
Date: October 3rd, 2016, Time 0.7540983606557377
Date: November 1st, 2016, Time 0.8333333333333334
Date: December 1st, 2016, Time 0.9153005464480874
Date: January 3rd, 2017, Time 1.0054794520547945


In [30]:
# Heston process
v0 = volatility*volatility  # spot variance
kappa = 0.1
theta = v0
sigma = 0.1
rho = -0.75

spot_handle = QuoteHandle(
    SimpleQuote(spot_price)
)
flat_ts = YieldTermStructureHandle(
    FlatForward(calculation_date, risk_free_rate, day_count)
)
dividend_yield = YieldTermStructureHandle(
    FlatForward(calculation_date, dividend_rate, day_count)
)
heston_process = HestonProcess(flat_ts,
                                dividend_yield,
                                spot_handle,
                                v0,
                                kappa,
                                theta,
                                sigma,
                                rho)

In [31]:
rng = GaussianRandomSequenceGenerator(
            UniformRandomSequenceGenerator(2 * (len(times)), UniformRandomGenerator()))
mpg = GaussianMultiPathGenerator(heston_process, times, rng, False)

In [32]:
sample_path = mpg.next()
multipath = sample_path.value()
for i in range(len(multipath)):
    print ("Time: {time}, Value: {value}, Vol: {vol}"
           .format(time=multipath[0].time(i), value=multipath[0].value(i), vol=multipath[1].value(i)))

Time: 0.0, Value: 1000.0, Vol: 0.04000000000000001
Time: 0.00819672131147541, Value: 1024.6890296726735, Vol: 0.0386494160612648
Time: 0.08469945355191257, Value: 1109.9198338838826, Vol: 0.03820133788746474
Time: 0.16393442622950818, Value: 1054.6415666622302, Vol: 0.039764647168185584
Time: 0.24863387978142076, Value: 963.5770346888274, Vol: 0.047322703042960525
Time: 0.3333333333333333, Value: 1032.0557124306752, Vol: 0.05436012075111825
Time: 0.41530054644808745, Value: 956.4997124452436, Vol: 0.06071361424498247
Time: 0.4972677595628415, Value: 968.1800452316464, Vol: 0.06602564884426856
Time: 0.5819672131147541, Value: 829.4760680149531, Vol: 0.07215254566568699
Time: 0.6666666666666666, Value: 786.3968115580153, Vol: 0.07332663854601808
Time: 0.7540983606557377, Value: 843.493667784652, Vol: 0.07859691070158777
Time: 0.8333333333333334, Value: 941.9242032841039, Vol: 0.06774613716439204
Time: 0.9153005464480874, Value: 974.0807026546613, Vol: 0.05638968252510732
Time: 1.00547945

In [33]:
def generate_paths(numpaths, timesteps):
    paths = [0]*numpaths
    for i in range(numpaths):
        sample_path = mpg.next()
        multipath = sample_path.value()
        time = [multipath[0].time(j) for j in range(len(multipath))]
        value = [multipath[0].value(j) for j in range(len(multipath))]
        vol = [multipath[1].value(j) for j in range(len(multipath))]
        paths[i] = list(zip(time, value, vol))
    return paths

In [34]:
path = generate_paths(10000, len(times))

In [35]:
def cliquet_average_return(paths, local_floor, local_cap, global_floor, global_cap):
    avg = [0]*len(paths)
    i=0
    for path in paths:
        path_sum = 0
        for time, index, vol in path:
            if time>0:
                path_sum += min(max(index/index_prior-1, local_floor), local_cap)
            index_prior = index
        path_sum = min(max(path_sum, global_floor), global_cap)
        avg[i]=path_sum
        i +=1
    index_return = sum(avg) / float(len(avg))
    return(index_return)

In [36]:
cliquet_average_return(path, -1000,.05, 0, 1000)

0.0353431033868719

In [37]:
flat_ts

<QuantLib.QuantLib.YieldTermStructureHandle; proxy of <Swig Object of type 'Handle< YieldTermStructure > *' at 0x0000020318DCF420> >